# 크롤링 전 설정

In [1]:
from selenium import webdriver

from itertools import product # 순열조합 만들어주는 패키지(에서 불러온 함수)
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from selenium.webdriver.common.keys import Keys # 키가 클릭되지 않을 때나 말 안들을 때 그냥 이거 쓰면 돼. # 이게 send_keys(Keys.ENTER) 쓰기 위해 불러오는 패키지임

import pandas as pd #데이터 내보내기 위해서 사용하는 패키지

In [16]:
# 버튼 알아 보는 단축키 ctrl + shift + c
# 셀 분할 ctrl + shift + -
# 셀 병합 shift + M

# 이거는 for문을 위한 바탕
issue = [1]
event = [1]
news = list(range(1,6)) # path 잘 살펴보면 규칙이 있는데 그 단계가 1~5로 돼 있음.
enter = [1]
sports = [1]
ages = list(range(1,7)) # 그 단계가 1~6으로 돼 있음.

condition = list(product(issue, event, news, enter, sports, ages))
# 데이터는 일단 많을 수록 나중에 좋으니까... 연령을 언제 또 나눠서 살펴 보고 싶을지 모르니까..

# 모든(?) 크롤링의 시작
driver = webdriver.Chrome('chromedriver')

# 네이버 데이터랩에서 크롤링 할거임.

Year = ['2020'] # 21년 전까지는 계속 2020, 21년 자료는 새로 만들어서 나중에 합쳐줘.

Mth = list(range(1,13,1)) # 또 문제네... int형은 숫자 하나일 때 01 이렇게 두자리수 표현이 안되넹.. 문자형으로 ㄱㄱ -->이건 일일이 해야하는데..? 그건 싫은데..?
M = [] # 이게 실제 사용할 거임~
for m in Mth:
    if len(str(m)) == 1:
        mm = '0' + str(m)
        M.append(mm)
    else:
        M.append(str(m))

Day = list(range(1,32))
D = [] # 이게 실제 사용할 거임~
for d in Day:
    if len(str(d)) == 1:
        dd = '0' + str(d)
        D.append(dd)
    else:
        D.append(str(d))



# 20년도 2월은 29일까지 존재,
# 4, 6, 9, 11월은 30일까지 존재,
# 1, 3, 5, 7, 8, 10, 12월은 31일까지 존재.

# 작업 순서
# 월일을 먼저 product 해주고 조건에 맞게 변형한 후, 년을 product 하는게 맞는 것 같으니 확인.

YMD = list(product(Year, M, D))
# YMD
# int(YMD[1][2]) #일 #튜플에서 인덱싱이 가능함을 확인.
# YMD[1][1] #월

# len(YMD)


del_lst = [] # boolean까지는 필요 없었음. index도 아니고 삭제할 요소 모아서 한번에 remove로 삭제.
for i in range(len(YMD)):
    if int(YMD[i][1]) == 2:
        if int(YMD[i][2]) in list(range(30,32)):
            del_lst.append(YMD[i])
    elif int(YMD[i][1]) in [4,6,9,11]:
        if int(YMD[i][2]) in [31]:
            del_lst.append(YMD[i])
# 년도까지 해서 6년 주기로 2월에 29일 넣어주는 것도 해주고 싶지만 일단 참자..

for j in del_lst:
    YMD.remove(j)

# 리스트 요소 삭제 (인덱스로는 안되는 것인지..)
# https://codetorial.net/tips_and_examples/remove_element_in_list_or_array.html
# https://jinisbonusbook.tistory.com/34


Hour = list(range(0,24)) # 0 ~ 23시
H = [] # 이게 실제 사용할 거임~
for h in Hour:
    if len(str(h)) == 1:
        hh = '0' + str(h)
        H.append(hh)
    else:
        H.append(str(h))

# Min = list(range(0,60)) # 0 ~ 59분
# Mn = [] # 이게 실제 사용할 거임~
# for mn in Min:
#     if len(str(mn)) == 1:
#         mnmn = '0' + str(mn)
#         Mn.append(mnmn)
#     else:
#         Mn.append(str(mn))

# Sec = 굳이 안정해줘도 됨, 걍 00임.

# 시는 따로 해줘야해...
# datetime 전에 YMD 이미 만들어 뒀잖슴. 그거 쓰는 거임.
datetime = YMD # 시는 마지막에 붙여줘야함. 심지어 앞에 T, 뒤에 %3A... # 뒤에 %3A는 안해줘도 됨. 일을 왜 굳이 두 번 하려고 해..

dt = 0
while dt < 15:
    del datetime[0]
    dt += 1

dtt = []
for i in range(len(datetime)):
    d = ",".join(list(datetime[i])).replace(",","-")
    dtt.append(d)

#dtt랑 시 조합 전에 시 앞에 T 해주기.. (문자열 덧셈으로!!!!!)
HH = []
for i in range(len(H)):
    h = 'T' + H[i]
    HH.append(h)

dtt2 = list(product(dtt, HH)) #dtt랑 시 조합 


dtt3 = []
for ii in range(len(dtt2)):
    dd = ",".join(list(dtt2[ii])).replace(",","")
    dtt3.append(dd)

dt = 0
while dt < 6:
    del dtt3[0]
    dt += 1
    
lst = []
lst_large = [] #for문 밖에서 해줘야 excel 파일 만들 수 있음.

for ii in range(len(dtt3)):
    print(f'{ii}. 일시: {dtt3[ii]}')
    driver.get(f'https://datalab.naver.com/keyword/realtimeList.naver?datetime={dtt3[ii]}%3A00%3A00')
    # 이건 렉 안먹기 위한 로딩 완료 기다리는 시간
    driver.implicitly_wait(10)

    # 버튼 클릭 코드
    # driver.find_element_by_xpath('').send_keys(Keys.ENTER)

    driver.find_element_by_xpath(f'//*[@id="content"]/div/div[2]/div[2]/div[1]/div[1]/div/div/ul/li[1]/div/div/a[1]').send_keys(Keys.ENTER) # issue
    driver.find_element_by_xpath(f'//*[@id="content"]/div/div[2]/div[2]/div[1]/div[1]/div/div/ul/li[2]/div/div/a[1]').send_keys(Keys.ENTER) # event
    time.sleep(0.5)
    driver.find_element_by_xpath(f'//*[@id="content"]/div/div[2]/div[2]/div[1]/div[1]/div/div/ul/li[4]/div/div/a[1]').send_keys(Keys.ENTER) # ent
    time.sleep(0.5)
    driver.find_element_by_xpath(f'//*[@id="content"]/div/div[2]/div[2]/div[1]/div[1]/div/div/ul/li[5]/div/div/a[1]').send_keys(Keys.ENTER) # sport
    time.sleep(0.5)
    
    for cond, j in zip(condition, range(1,len(condition)+1)):
        driver.find_element_by_xpath(f'//*[@id="content"]/div/div[2]/div[2]/div[1]/div[1]/div/div/ul/li[3]/div/div/a[{cond[2]}]').send_keys(Keys.ENTER) # news
        driver.implicitly_wait(1)
        driver.find_element_by_xpath(f'//*[@id="content"]/div/div[2]/div[1]/div[2]/div/div/div/ul/li[{cond[5]}]/a').send_keys(Keys.ENTER) # age
        driver.implicitly_wait(1)

        print(f'{j}. 설정이 {cond}일 때')
        # for문 안에서 실검 가져오기
        rank1_10 = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[2]/div[2]/div/div/ul[1]')
        rank11_20 = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[2]/div[2]/div/div/ul[2]') # /li[1]/div/span[2]/span #이건 11~20위
        rank1_10_list = rank1_10.find_elements_by_tag_name('li')
        rank11_20_list = rank11_20.find_elements_by_tag_name('li')

        for i in range(len(rank1_10_list)):
            name = rank1_10_list[i].text
            name = name.replace('\n',' ')
            name = name[2:] # 숫자 없애고 싶어서 추가 함. 이거 빼면 숫자도 나옴.
            idx = name.find('위') + 1

            if '연관검색어' in name:
                relate_idx = name.find('연관검색어')
                lst.append(name[idx:relate_idx])
            else:  
                lst.append(name[idx:])
            time.sleep(1)
        
        for i in range(len(rank11_20_list)):
            name = rank11_20_list[i].text
            name = name.replace('\n',' ')
            name = name[2:] # 숫자 없애고 싶어서 추가 함. 이거 빼면 숫자도 나옴.
            idx = name.find('위') + 1

            if '연관검색어' in name:
                relate_idx = name.find('연관검색어')
                lst.append(name[idx:relate_idx])
            else:  
                lst.append(name[idx:])
            time.sleep(1)

        lst
        lst_large.append(lst)
        
        lst = [] # lst 초기화...

        time.sleep(1)
    
    # time.sleep(시간)은 무조건 기다려야 하는 코드,
    # 객체.implicitly_wait(시간)은 작동? 완료 되면 그냥 무시되는 코드

0. 일시: 2020-01-16T06
1. 설정이 (1, 1, 5, 1, 1, 6)일 때


['고민정',
 '배드파더스',
 '이국종 교수',
 '이국종',
 '에이치엘비',
 '주예지',
 '연말정산 간소화 서비스',
 '인플루언서',
 '연말정산',
 ' 다우지수',
 ' 미국증시',
 ' 뉴욕증시',
 ' 택배조회',
 ' 미중 무역협상',
 ' 간첩',
 ' 김건모',
 ' 시동',
 ' 한국시간',
 ' 야채부락리',
 ' 라비앙']

1. 일시: 2020-01-16T07
1. 설정이 (1, 1, 5, 1, 1, 6)일 때


['고민정',
 '이해찬',
 '이낙연',
 '파키스탄',
 '배드파더스',
 '이국종 교수',
 '주예지',
 '일출시간',
 '국세청 연말정산 간소화',
 ' 이국종',
 ' 연말정산 간소화 서비스',
 ' 다우지수',
 ' 뉴욕증시',
 ' 연말정산',
 ' 미국증시',
 ' 미중 무역협상',
 ' 나스닥',
 ' 에이치엘비생명과학',
 ' 네이버증권',
 ' 김건모']

In [17]:
# 엑셀(excel)로 받기
#lst_large
raw_data = pd.DataFrame(lst_large)
colnms = range(1,21)#range(1,11)
#index(rownames)는 condition_sample 사용

raw_data.columns = list(colnms) #colnms 변경
r_ind = list(product(dtt3, condition))
raw_data.index = r_ind #rownms 변경
raw_data
raw_data.to_excel(excel_writer= '2020 실검 1~20위.xlsx') #엑셀로 저장

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
"(2020-01-16T06, (1, 1, 5, 1, 1, 6))",고민정,배드파더스,이국종 교수,이국종,에이치엘비,주예지,연말정산 간소화 서비스,인플루언서,연말정산,다우지수,미국증시,뉴욕증시,택배조회,미중 무역협상,간첩,김건모,시동,한국시간,야채부락리,라비앙
"(2020-01-16T07, (1, 1, 5, 1, 1, 6))",고민정,이해찬,이낙연,파키스탄,배드파더스,이국종 교수,주예지,일출시간,국세청 연말정산 간소화,이국종,연말정산 간소화 서비스,다우지수,뉴욕증시,연말정산,미국증시,미중 무역협상,나스닥,에이치엘비생명과학,네이버증권,김건모
